In [1]:
import numpy as np
from hmmlearn import hmm
import csv


In [2]:
hmm_states = 2

model = hmm.CategoricalHMM(n_components=hmm_states)
model.emissionprob_ = np.array([[0.1,0.9],[0.9,0.1]])
model.transmat_ = np.array([[0.9,0.1],[0.1,0.9]])
model.startprob_ = np.array([0.8,0.2])
data,states = model.sample(n_samples = 1000,random_state=28)
data = np.squeeze(data)

In [3]:

# constant n_obs
# n_obs = 1000
# # Set p(H) apriori for simulation as a biased coin.
# p_h = 0.8

# # Model experiment as a single biased coin flipped 1000 times.
# data = np.random.binomial(1, p_h, n_obs=n_obs)

In [4]:
n_states = 2
n_features = 2
n_obs = data.shape[0]
# emission = np.random.rand(n_obs,n_states)
# emission = emission/np.tile(np.expand_dims(np.sum(emission,axis=1),axis=1),(1,2))

emission = np.random.rand(n_states,n_features)
emission = emission/np.tile(np.expand_dims(np.sum(emission,axis=1),axis=1),(1,2))
# emission = np.array([[0.5,0.5],[0.5,0.5]])

transition =  np.random.rand(n_states,n_states)
transition = transition/np.tile(np.expand_dims(np.sum(transition,axis=1),axis=1),(1,2))
# transition = np.array([[0.6,0.4],[0.5,0.5]])


forward = np.random.rand(n_obs,n_states)
scale_factors = np.zeros((n_obs))
forward_hat = np.zeros((n_obs,n_states))

backward = np.random.rand(n_obs,n_states)
backward_hat = np.zeros((n_obs,n_states))

init_prob = np.array([0.5,0.5])


p_old = -10000
tol = 0.0001
max_iter = 100

mu = np.random.rand(n_states)

In [5]:
print(f'emission is {emission} and transition is {transition}')

emission is [[0.92747916 0.07252084]
 [0.57271225 0.42728775]] and transition is [[0.33003206 0.66996794]
 [0.68301413 0.31698587]]


In [6]:
for ite in range(max_iter):
    forward_hat = np.zeros((n_obs,n_states))
    backward_hat = np.zeros((n_obs,n_states))
    scale_factors = np.zeros((n_obs))

    for i in range(n_states):
        forward[0,i] = init_prob[i] * emission[i,data[0]]
    scale_factors[0] = np.sum(forward[0,:])
    forward_hat[0,:] = forward[0,:]/scale_factors[0]
    
    for t in range(n_obs-1):
        for i in range(n_states):
            for j in range(n_states):
                forward_hat[t+1,i] += forward_hat[t,j]*transition[j,i]
                # temp = np.matmul(forward_hat[t,i] ,np.transpose(transition[i,j])) * emission[i,data[t+1]]
            forward_hat[t+1,i] = forward_hat[t+1,i]*emission[i,data[t+1]]
        scale_factors[t+1] = np.sum(forward_hat[t+1,:])

        forward_hat[t+1,:] = forward_hat[t+1,:]/scale_factors[t+1]
        # print(f'temp is {temp} and the scale factor is {scale_factors[t+1]} and the forward_hat is {forward_hat[t+1]}')

        forward[t+1,i] = forward_hat[t+1,:]*np.prod(scale_factors[0:t+1])

    backward[-1,:] = 1
    backward_hat[-1,:] = backward[-1,:] * scale_factors[-1]
    for t in reversed(range(n_obs-1)):
        temp = np.matmul(backward_hat[t+1,:]*emission[:,data[t+1]],transition)
        backward_hat[t,:] = temp/scale_factors[t]
        backward[t,:] = backward_hat[t,:]*np.prod(scale_factors[t+1:-1])


    a = np.zeros((n_obs,n_states))
    b = np.zeros((n_obs,n_states,n_states))
    for i in range(n_obs):
        for j in range(n_states):
            a[i,j] = forward_hat[i,j]*backward_hat[i,j]
    for t in range(n_obs-1):
        for i in range(n_states):
            for j in range(n_states):
                b[t,i,j] = scale_factors[t+1]*forward_hat[t,i]*backward_hat[t+1,j]*transition[i,j]*emission[j,data[t+1]]

    for i in range(n_states):
        for j in range(n_states):
            transition[i,j] = np.sum(b[0:-1,i,j])/np.sum(b[0:-1,i,:])
            # print(np.sum(b[0:-1,i,j]),np.sum(b[0:-1,i,:]))

    for i in range(n_states):
        init_prob[i] = a[0,i]/np.sum(a[0,:])

    for i in range(n_states):
        for j in range(n_states):
            emission[j,i] = np.sum(a[np.argwhere(data==i),j]) / np.sum(a[:,j])
            
    p = np.sum(np.log(scale_factors))
    print(f'p is:{p},transition is {transition},emission is {emission},init is {init_prob}')
    # print(f'transition prob is: {transition}')
    # print(f'emission prob is:{emission}')
    if p>p_old and p - p_old < tol:
        break
    p_old = p

p is:-905.2948059895389,transition is [[0.37103854 0.62896146]
 [0.67008749 0.32991251]],emission is [[0.41469972 0.58530028]
 [0.12722988 0.87277012]],init is [0.20929163 0.79070837]
p is:-821.2189721244741,transition is [[0.39278861 0.60721139]
 [0.65831065 0.34168935]],emission is [[0.81766958 0.18233042]
 [0.577769   0.422231  ]],init is [0.17185899 0.82814101]
p is:-802.0325478130133,transition is [[0.42745744 0.57254256]
 [0.66831028 0.33168972]],emission is [[0.39416959 0.60583041]
 [0.20595311 0.79404689]],init is [0.10364962 0.89635038]
p is:-773.3675729770832,transition is [[0.467065   0.532935  ]
 [0.69009448 0.30990552]],emission is [[0.75516685 0.24483315]
 [0.59630134 0.40369866]],init is [0.09795041 0.90204959]
p is:-777.8328842473427,transition is [[0.53768313 0.46231687]
 [0.73887191 0.26112809]],emission is [[0.37145174 0.62854826]
 [0.24558186 0.75441814]],init is [0.0855567 0.9144433]
p is:-756.3963366999649,transition is [[0.64927839 0.35072161]
 [0.81419367 0.1858

In [7]:
# model = hmm.CategoricalHMM(n_components=n_states,emissionprob_prior=emission,transmat_prior=transition,startprob_prior=init_prob)
# model2 = hmm.CategoricalHMM(n_components=hmm_states,n_iter=1000)


In [8]:
# model2.fit(data.reshape(-1,1))
# hidden_states = model2.predict(data.reshape(-1,1))
# print("Most likely hidden states:", hidden_states)


In [9]:
# model2.monitor_.converged


In [10]:
# model2.emissionprob_

In [11]:
# model2.transmat_